# 課題1

In [219]:
class TicTacToe:
    def __init__(self):
        self.players = ['X', 'O']
        
    def is_terminal(self, state):
        if self.is_draw(state):
            return True
        elif self.is_win(state,0) or self.is_win(state,1):
            return True
        return False
        
    def is_draw(self, state):
        if self.is_win(state,0) or self.is_win(state,1):
            return False
        for i in state:
            for j in i:
                if j==' ':
                    return False
        return True

    def is_win(self, state, player_id):
        a=self.players[player_id]
        li=[]
        for i in range(3):
            if state[i]==[a,a,a]:
                return True
            for j in range(3):
                if state[i][j]==a:
                    li.append([i,j])
        pat=[[[i,j] for i in [0,1,2]] for j in [0,1,2]] +[[[0,0],[1,1],[2,2]]]+[[[0,2],[1,1],[2,0]]]
        for i in pat:
            x=0
            for j in i:
                if j in li:
                    x+=1
            if x==3:
                return True
        return False
    
    def reward(self,state):
        if self.is_draw(state):
            return 0
        elif self.is_win(state,0):
            return 1
        elif self.is_win(state,1):
            return -1
        
    def make_move(self, state, move, player_id):
        new_state = copy.deepcopy(state)
        x, y = move
        char = self.players[player_id]
        new_state[x][y] = char
        return new_state
    
    def board_blank(self,state):
        blank=[]
        for i in range(3):
            for j in range(3):
                if state[i][j]==' ':
                    blank.append([i,j])
        return blank

In [220]:
inf=10000000

class Node():
    def __init__(self,state,pn,dpn,child=[],ao=0,parent=None):
        self.type=["Or","And"]
        self.ao=ao
        self.pn=pn
        self.dpn=dpn
        self.child=child
        self.parent=parent
        self.state=state
    
    def select_child(self):
        if self.ao==0:
            return sorted(self.child, key=lambda u: u.pn)[0]
        else:
            return sorted(self.child, key=lambda u: u.dpn)[0]
            
    def back(self):
        if self.ao==0:
            self.pn=self.select_child().pn
            self.dpn=sum([i.dpn for i in self.child])
        else:
            self.dpn=self.select_child().dpn
            self.pn=sum([i.pn for i in self.child])
            
    

In [221]:
def provesearch_1(root_state):
    #引き分けを先手の勝ち
    game=TicTacToe()
    root = Node(root_state,ao=0,pn=1,dpn=1)
    stop_flag=1
    while stop_flag:
        state_node=root
        while state_node.child!=[]:
            state_node=state_node.select_child()
        
        if not game.is_terminal(state_node.state):
            blank=game.board_blank(state_node.state)
            for i in blank:
                state=game.make_move(state_node.state,i,state_node.ao)
                if not game.is_terminal(state):
                    tem_node=Node(state,pn=1,dpn=1,child=[],ao=1-state_node.ao,parent=state_node)
                    state_node.child.append(tem_node)
                else:
                    reward=game.reward(state)
                    if reward != -1:
                        tem_node=Node(state,pn=0,dpn=inf,child=[],ao=1-state_node.ao,parent=state_node)
                        state_node.child.append(tem_node)
                    else:
                        tem_node=Node(state,pn=inf,dpn=0,child=[],ao=1-state_node.ao,parent=state_node)
                        state_node.child.append(tem_node)
                        
            while(state_node.parent != None):
                state_node.back()
                state_node=state_node.parent
        else:
            stop_flag=0
    
    root.back()
    return (root.pn, root.dpn),root

In [222]:
import copy
root_state= [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]
result,root=provesearch_1(root_state)
print("result:",result)

result: (0, 10000064)


In [223]:
inf=10000000

class Node():
    def __init__(self,state,pn,dpn,child=[],ao=0,parent=None):
        self.type=["Or","And"]
        self.ao=ao
        self.pn=pn
        self.dpn=dpn
        self.child=child
        self.parent=parent
        self.state=state
    
    def select_child(self):
        if self.ao==0:
            return sorted(self.child, key=lambda u: u.pn)[0]
        else:
            return sorted(self.child, key=lambda u: u.dpn)[0]
            
    def back(self):
        if self.ao==0:
            self.pn=self.select_child().pn
            self.dpn=sum([i.dpn for i in self.child])
        else:
            self.dpn=self.select_child().dpn
            self.pn=sum([i.pn for i in self.child])
            
    

In [224]:
def provesearch_2(state):
    #引き分けを後手の勝ち
    game=TicTacToe()
    root = Node(state,ao=0,pn=1,dpn=1)
    stop_flag=1
    while stop_flag:
        state_node=root
        while state_node.child!=[]:
            state_node=state_node.select_child()
        
        if not game.is_terminal(state_node.state):
            blank=game.board_blank(state_node.state)
            for i in blank:
                state=game.make_move(state_node.state,i,state_node.ao)
                if not game.is_terminal(state):
                    tem_node=Node(state,pn=1,dpn=1,child=[],ao=1-state_node.ao,parent=state_node)
                    state_node.child.append(tem_node)
                else:
                    reward=game.reward(state)
                    if reward != 1:
                        tem_node=Node(state,pn=inf,dpn=0,child=[],ao=1-state_node.ao,parent=state_node)
                        state_node.child.append(tem_node)
                    else:
                        tem_node=Node(state,pn=0,dpn=inf,child=[],ao=1-state_node.ao,parent=state_node)
                        state_node.child.append(tem_node)
                        
            while(state_node.parent != None):
                state_node.back()
                state_node=state_node.parent
        else:
            stop_flag=0

    root.back()
    return (root.pn, root.dpn),root

In [225]:
root_state= [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]
result,root=provesearch_2(root_state)
print("result:",result)

result: (10000007, 0)


# 課題2

In [29]:
inf=10000000

class Node():
    def __init__(self,state,pn,dpn,child=[],ao=0,parent=None):
        self.type=["Or","And"]
        self.ao=ao
        self.pn=pn
        self.dpn=dpn
        self.child=child
        self.parent=parent
        self.state=state
    
    def select_child(self):
        if self.ao==0:
            return sorted(self.child, key=lambda u: u.pn)[0]
        else:
            return sorted(self.child, key=lambda u: u.dpn)[0]
            
    def back(self):
        if self.ao==0:
            self.pn=self.select_child().pn
            self.dpn=sum([i.dpn for i in self.child])
        else:
            self.dpn=self.select_child().dpn
            self.pn=sum([i.pn for i in self.child])
            
    

In [30]:
class Domineering:
    def __init__(self,size):
        self.players = ['R', 'B']
        self.size=size
    
    def board_blank(self,state):
        blank=[]
        for i in range(self.size):
            for j in range(self.size):
                if i+1<self.size and state[i+1][j]==" ":
                    blank.append([[i,j],[i+1,j]])
                if j+1<self.size and state[i][j+1]==" ":
                    blank.append([[i,j],[i,j+1]])
        return blank
                
    def make_move(self, state, move, player_id):
        new_state = copy.deepcopy(state)
        x, y = move
        char = self.players[player_id]
        new_state[x[0]][x[1]] = char
        new_state[y[0]][y[1]] = char
        return new_state


In [31]:
def provesearch(root_state,size,itera):
    game = Domineering(size)
    root = Node(root_state,ao=0,pn=1,dpn=1)
    for ite in range(itera):
        state_node=root
        while state_node.child!=[]:
            state_node=state_node.select_child()
        
        if not game.board_blank(state_node.state)==[]:
            blank=game.board_blank(state_node.state)
            for i in blank:
                state=game.make_move(state_node.state,i,state_node.ao)
                if game.board_blank(state)==[]:
                    if state_node.ao==0:
                        tem_node=Node(state,pn=0,dpn=inf,child=[],ao=1-state_node.ao,parent=state_node)
                        state_node.child.append(tem_node)
                    else:
                        tem_node=Node(state,pn=inf,dpn=0,child=[],ao=1-state_node.ao,parent=state_node)
                        state_node.child.append(tem_node)
                else:
                    tem_node=Node(state,pn=1,dpn=1,child=[],ao=1-state_node.ao,parent=state_node)
                    state_node.child.append(tem_node)
                        
            while(state_node.parent != None):
                state_node.back()
                state_node=state_node.parent
    
    root.back()
    return (root.pn, root.dpn),root

In [32]:
import copy
root_state= [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]
result,root=provesearch(root_state,size=3,itera=1000)
print("result:",result)
# for i in root.child:
#     print("------")
#     print(i.state)
#     print(i.pn)
#     print(i.dpn)

result: (10000001, 0)


In [33]:
inf=10000000

class Node():
    def __init__(self,state,pn,dpn,child=[],ao=0,parent=None):
        self.type=["Or","And"]
        self.ao=ao
        self.pn=pn
        self.dpn=dpn
        self.child=child
        self.parent=parent
        self.state=state
    
    def select_child(self):
        if self.ao==0:
            return sorted(self.child, key=lambda u: u.pn)[0]
        else:
            return sorted(self.child, key=lambda u: u.dpn)[0]
            
    def back(self):
        if self.ao==0:
            self.pn=self.select_child().pn
            self.dpn=sum([i.dpn for i in self.child])
        else:
            self.dpn=self.select_child().dpn
            self.pn=sum([i.pn for i in self.child])
            

root_state= [ [' ',' ',' ',' '],
              [' ',' ',' ',' '],
              [' ',' ',' ',' '],
              [' ',' ',' ',' ']]
result,root=provesearch(root_state,size=4,itera=10000)
print("result:",result)
# for i in root.child:
#     print("------")
#     print(i.state)
#     print(i.pn)
#     print(i.dpn)

result: (180, 436)


In [34]:
inf=10000000

class Node():
    def __init__(self,state,pn,dpn,child=[],ao=0,parent=None):
        self.type=["Or","And"]
        self.ao=ao
        self.pn=pn
        self.dpn=dpn
        self.child=child
        self.parent=parent
        self.state=state
    
    def select_child(self):
        if self.ao==0:
            return sorted(self.child, key=lambda u: u.pn)[0]
        else:
            return sorted(self.child, key=lambda u: u.dpn)[0]
            
    def back(self):
        if self.ao==0:
            self.pn=self.select_child().pn
            self.dpn=sum([i.dpn for i in self.child])
        else:
            self.dpn=self.select_child().dpn
            self.pn=sum([i.pn for i in self.child])
            

root_state= [ [' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ']]
result,root=provesearch(root_state,size=5,itera=10000)
print("result:",result)

result: (125, 1306)
